In [2]:
import ujson as json
import numpy as np
import pandas as pd
import openai
import collections
import time

In [3]:
# File definition and load
test_file = './data/test_revised.json'
relation_file = './data/relations.txt'

with open(test_file, "r") as fh:
    data = json.load(fh)
    

In [4]:

# def vertex_process(vertex_set):
    
#     vertex = [item['name'] for item in vertex_set[0]]
# #     print(vertex[1])
    
def relation_triple(labels, vertex_set,):
    triples = []
    head_entity = []
    tail_entity = []
    relation_labels = []
    for sample in labels:
        head_entity.append(vertex_set[sample['h']][0]['name'])
        tail_entity.append(vertex_set[sample['t']][0]['name'])
        relation_labels.append(rel_dict[sample['r']])
            
    triples = list(zip(head_entity,tail_entity,relation_labels))
    return triples

'''
[(0, 6, 'P577'), (0, 2, 'P175'), (10, 8, 'P131'), (8, 7, 'P17'), (10, 7, 'P17'), (2, 1, 'P27'), (8, 5, 'P30'), (0, 14, 'P577'), (2, 0, 'P800'), (8, 7, 'P131'), (10, 7, 'P131')]
'''

rel_dict = collections.defaultdict(int)

# Display all the relations. 
def relations_design(relation_file, rel_dict):
    with open(relation_file,'r') as rf:
        relations = rf.readlines()
    
    output_rel_grp = []
    count = 0
    output_rel = ''
    for r in relations:
        

        # Split the lable and relation
        label = r.split(' ', 1)[0]
        relation = r.split(' ', 1)[1][:-1]
        
        # Form the dictionary of label and relation
        rel_dict[label] = relation
        
        # Generate the relation for prompt
        temp = '- ' + relation +'\n'
        output_rel +=   temp
        count += 1
#         print("output_rel==============\n"+output_rel)
        if count %10 == 6:
            output_rel_grp.append(output_rel)
#             print("temp==============\n"+output_rel)
            
            output_rel = ''
            
            
    return output_rel_grp

# rel_in_prompt = relations_design(relation_file, rel_dict)


def preprocess(article_data):
    
    # Read the conponents
    
    title = article_data['title']
    vertex_set = article_data['vertexSet']
    sentence = article_data['sents']
    labels = article_data['labels']
    
    
    # Paragraph Form -> Insight 1: Robustness of sentence.
    sentence_list = [' '.join(s) for s in sentence]
    paragraph = ' '.join(sentence_list)


    # Process the relation and generate the triples
    
    rel_in_prompt = relations_design(relation_file, rel_dict)
    triples = relation_triple(labels, vertex_set)
    return triples, rel_in_prompt, paragraph



In [5]:

# Different Instructions
def instructions_design():
    
    instruction = collections.defaultdict(int)
    
    instruction['baseline'] = 'Instruction: the paragraph is from an article of Wikipedia. Category the relation between the entity "<SEP>" and entity "<SEP>" to one or more available values, the available values are:'
    instruction['reformulate'] = 'Instruction: the paragraph is from an article of Wikipedia. Specify the relations that exist between the entity "<SEP>" and entity "<SEP>" . Choose from the following relations:'
    instruction['attention'] = 'Attention: You are restricted to choose from the following relationships'
    instruction['order'] = 'Instruction: the paragraph is from an article of Wikipedia. Determine the relationship that exists from the entity "<SEP>" to the entity "<SEP>" . Choose strictly from the following relations:'
    return instruction

def prompt_design(head_entity,tail_entity, rel_in_prompt, paragraph):
    
    instruction = [instructions_design()['reformulate']][0].split("<SEP>")

#     concat_instruction = instruction[0]+head_entity+instruction[1]+tail_entity+instruction[2] + '\n' + instructions_design()['attention']+'\n'
    concat_instruction = instruction[0]+head_entity+instruction[1]+tail_entity+instruction[2] + '\n' 
    handcrafted_prompt = concat_instruction + rel_in_prompt + '\n"""\n' + paragraph + '\n"""'

    return handcrafted_prompt
                   

                   
                   

def gpt(my_prompt):
    

    
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=my_prompt,
        temperature=0.1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response

def relation_classification(triples, rel_in_prompt_group, paragraph):
    total = 0
    correct = 0
    for sample in triples:
        head_entity = sample[0]
        tail_entity = sample[1]
        relation = sample[2]
        output = ''
        for rel_in_prompt in rel_in_prompt_group:
        
            prompt = prompt_design(head_entity,tail_entity, rel_in_prompt, paragraph)
            print("Prompt==============")
            print(prompt)
            print("Output==============")
            
            output_relation = gpt(prompt)["choices"][0]["text"]
            print(output_relation)
            output+=output_relation
        
        print("The relation between "+ head_entity+ ' and '+ tail_entity+ ' is:')
        print(output)
        
        if relation in output:
            correct+=1
            
        total+=1
        
        print('-----------TIME-------------------')
        localtime = time.localtime()
        result = time.strftime("%I:%M:%S %p", localtime)
        print(result)
        print('-----------TIME-------------------')
        
        time.sleep(10)
        break
#         print('\n')
#         break
    return correct,total
    


In [6]:


# Form the preprocessed dataset.

def batch(data):
    total = 0
    correct = 0
    
    for i in range(len(data)):
        article_data = data[i]
        triples, rel_in_prompt_group, paragraph = preprocess(article_data)
        c,t = relation_classification(triples, rel_in_prompt_group, paragraph)
        correct += c
        total += t
        
        print("==========")
#         print(triples)
        break
    
#     precision = c / 
    print("The micro f1-score of the GPT-3 model is : "+ str(correct/total*100)+ "%")


batch(data)




Prompt==============
Instruction: the paragraph is from an article of Wikipedia. Specify the relations that exist between the entity "Loud" and entity "2010" . Choose from the following relations:
- head of government
- country
- place of birth
- place of death
- father
- mother

"""
The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth studio album Loud ( 2010 ) . Critics acclaimed the show for its liveliness and higher caliber of quality when compared to Rihanna 's previous tours . The Loud Tour was a large commercial success , experiencing demand for an extension of shows in the United Kingdom due to popularity . In London , Rihanna played a record breaking 10 dates at The O2 Arena . The tour ultimately grossed an estimated value of US$ 90 million from 98 reported shows and a total audience of 1,200,800 . The Loud T



Relations: original network - Loud (2010); has part - Loud Tour (2010)
Prompt==============
Instruction: the paragraph is from an article of Wikipedia. Specify the relations that exist between the entity "Loud" and entity "2010" . Choose from the following relations:
- date of death
- inception
- dissolved, abolished or demolished
- publication date
- start time
- end time
- point in time
- conflict
- characters
- lyrics by

"""
The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth studio album Loud ( 2010 ) . Critics acclaimed the show for its liveliness and higher caliber of quality when compared to Rihanna 's previous tours . The Loud Tour was a large commercial success , experiencing demand for an extension of shows in the United Kingdom due to popularity . In London , Rihanna played a record breaking 10 dates at

In [7]:


# Form the preprocessed dataset.

def batch(data):
    total = 0
    correct = 0
    
    for i in range(len(data)):
        article_data = data[i]
        triples, rel_in_prompt_group, paragraph = preprocess(article_data)
        c,t = relation_classification(triples, rel_in_prompt_group, paragraph)
        correct += c
        total += t
#     re_p = 1.0 * correct_re / len(submission_answer)
#     re_r = 1.0 * correct_re / tot_relations
#     if re_p+re_r == 0:
#         re_f1 = 0
#     else:
#         re_f1 = 2.0 * re_p * re_r / (re_p + re_r)
        print("==========")
#         print(triples)
        break
    print("The micro f1-score of the GPT-3 model is : "+ str(correct/total*100)+ "%")


batch(data)




Prompt==============
Instruction: the paragraph is from an article of Wikipedia. Specify the relations that exist between the entity "Loud" and entity "2010" . Choose from the following relations:
- head of government
- country
- place of birth
- place of death
- father
- mother

"""
The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth studio album Loud ( 2010 ) . Critics acclaimed the show for its liveliness and higher caliber of quality when compared to Rihanna 's previous tours . The Loud Tour was a large commercial success , experiencing demand for an extension of shows in the United Kingdom due to popularity . In London , Rihanna played a record breaking 10 dates at The O2 Arena . The tour ultimately grossed an estimated value of US$ 90 million from 98 reported shows and a total audience of 1,200,800 . The Loud T



Relations: original network - Loud (2010); has part - Loud Tour (2010)
Prompt==============
Instruction: the paragraph is from an article of Wikipedia. Specify the relations that exist between the entity "Loud" and entity "2010" . Choose from the following relations:
- date of death
- inception
- dissolved, abolished or demolished
- publication date
- start time
- end time
- point in time
- conflict
- characters
- lyrics by

"""
The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth studio album Loud ( 2010 ) . Critics acclaimed the show for its liveliness and higher caliber of quality when compared to Rihanna 's previous tours . The Loud Tour was a large commercial success , experiencing demand for an extension of shows in the United Kingdom due to popularity . In London , Rihanna played a record breaking 10 dates at



Relations: follows, supported by, genre.
Prompt==============
Instruction: the paragraph is from an article of Wikipedia. Specify the relations that exist between the entity "Loud" and entity "Rihanna" . Choose from the following relations:
- producer
- award received
- creator
- parent taxon
- ethnic group
- performer
- manufacturer
- developer
- series
- sister city

"""
The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth studio album Loud ( 2010 ) . Critics acclaimed the show for its liveliness and higher caliber of quality when compared to Rihanna 's previous tours . The Loud Tour was a large commercial success , experiencing demand for an extension of shows in the United Kingdom due to popularity . In London , Rihanna played a record breaking 10 dates at The O2 Arena . The tour ultimately grossed an estimated 

Prompt==============
Instruction: the paragraph is from an article of Wikipedia. Specify the relations that exist between the entity "The O2 Arena" and entity "London" . Choose from the following relations:
- head of government
- country
- place of birth
- place of death
- father
- mother

"""
The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth studio album Loud ( 2010 ) . Critics acclaimed the show for its liveliness and higher caliber of quality when compared to Rihanna 's previous tours . The Loud Tour was a large commercial success , experiencing demand for an extension of shows in the United Kingdom due to popularity . In London , Rihanna played a record breaking 10 dates at The O2 Arena . The tour ultimately grossed an estimated value of US$ 90 million from 98 reported shows and a total audience of 1,200,800 . 


Relations between "The O2 Arena" and "London":
- Has part
- Residence
Prompt==============
Instruction: the paragraph is from an article of Wikipedia. Specify the relations that exist between the entity "The O2 Arena" and entity "London" . Choose from the following relations:
- date of death
- inception
- dissolved, abolished or demolished
- publication date
- start time
- end time
- point in time
- conflict
- characters
- lyrics by

"""
The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth studio album Loud ( 2010 ) . Critics acclaimed the show for its liveliness and higher caliber of quality when compared to Rihanna 's previous tours . The Loud Tour was a large commercial success , experiencing demand for an extension of shows in the United Kingdom due to popularity . In London , Rihanna played a record breaking 10 

...


Relations:
- followed by: The Loud Tour
- genre: The Loud Tour
- operator: The Loud Tour
- contains administrative territorial entity: The Loud Tour (over twenty countries in the Americas and Europe)
Prompt==============
Instruction: the paragraph is from an article of Wikipedia. Specify the relations that exist between the entity "Rihanna" and entity "Loud" . Choose from the following relations:
- producer
- award received
- creator
- parent taxon
- ethnic group
- performer
- manufacturer
- developer
- series
- sister city

"""
The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth studio album Loud ( 2010 ) . Critics acclaimed the show for its liveliness and higher caliber of quality when compared to Rihanna 's previous tours . The Loud Tour was a large commercial success , experiencing demand for an extension of sho

Prompt==============
Instruction: the paragraph is from an article of Wikipedia. Specify the relations that exist between the entity "London" and entity "United Kingdom" . Choose from the following relations:
- head of government
- country
- place of birth
- place of death
- father
- mother

"""
The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth studio album Loud ( 2010 ) . Critics acclaimed the show for its liveliness and higher caliber of quality when compared to Rihanna 's previous tours . The Loud Tour was a large commercial success , experiencing demand for an extension of shows in the United Kingdom due to popularity . In London , Rihanna played a record breaking 10 dates at The O2 Arena . The tour ultimately grossed an estimated value of US$ 90 million from 98 reported shows and a total audience of 1,200,800 



Relations: 
- Member of: United Kingdom 
- Has part: London
Prompt==============
Instruction: the paragraph is from an article of Wikipedia. Specify the relations that exist between the entity "London" and entity "United Kingdom" . Choose from the following relations:
- date of death
- inception
- dissolved, abolished or demolished
- publication date
- start time
- end time
- point in time
- conflict
- characters
- lyrics by

"""
The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth studio album Loud ( 2010 ) . Critics acclaimed the show for its liveliness and higher caliber of quality when compared to Rihanna 's previous tours . The Loud Tour was a large commercial success , experiencing demand for an extension of shows in the United Kingdom due to popularity . In London , Rihanna played a record breaking 10 dates a



Relations:
- performed at: The O2 Arena, United Kingdom
Prompt==============
Instruction: the paragraph is from an article of Wikipedia. Specify the relations that exist between the entity "The O2 Arena" and entity "United Kingdom" . Choose from the following relations:
- owned by
- located in the administrative territorial entity
- genre
- operator
- religion
- contains administrative territorial entity
- follows
- followed by
- headquarters location
- cast member

"""
The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth studio album Loud ( 2010 ) . Critics acclaimed the show for its liveliness and higher caliber of quality when compared to Rihanna 's previous tours . The Loud Tour was a large commercial success , experiencing demand for an extension of shows in the United Kingdom due to popularity . In London , Ri



Relations:
- present in work (The O2 Arena and United Kingdom)
The relation between The O2 Arena and United Kingdom is:


Relations: country (United Kingdom) - The O2 Arena is located in the United Kingdom.
Relations: instance of, country of citizenship (United Kingdom)

Relations:
- performed at: The O2 Arena, United Kingdom

Relations: located in the administrative territorial entity (United Kingdom), contains administrative territorial entity (The O2 Arena)

Relations: performer, sister city

Relations: 
- located in or next to body of water: The O2 Arena is located in the United Kingdom.
- location: The O2 Arena is located in the United Kingdom.

Relations: 
- Country of Origin: United Kingdom 
- Has Part: The O2 Arena

Relations between the entity "The O2 Arena" and entity "United Kingdom":
- Inception: The O2 Arena is located in the United Kingdom.
- Start Time: The Loud Tour began in the United Kingdom with a show at The O2 Arena.
- End Time: The Loud Tour ended in the United 